In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
datapath =  'drive/My Drive/'

In [7]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv1D, Conv3D, Conv2DTranspose, Conv3DTranspose
from keras.models import Sequential, Model
from keras.optimizers import Adam, SGD
import keras
import os
from keras.losses import mean_squared_error, kullback_leibler_divergence
from keras import backend as K
import matplotlib.pyplot as plt
import cv2
import os
import binvox_rw
from shutil import copyfile
import sys
import numpy as np

DATA PRE PROCESSING





In [8]:


def gen_images():
    directory = "species2"
    out_dir = "./mask_images/"

    for root, dirs, files in os.walk(directory):
        for filename in files:
            # Hardcoding these to avoid mishaps if script runs again by mistake
            if filename == "1.png": 
                try:
                    print("root:",root,"dirs:",dirs,"files:",files)
                    normal = root+'\\1_crop.png'
                    #print("normal_image path:"+normal)
                    img1 = cv2.imread(root+'/1_crop.png')
                    img2 = cv2.imread(root+'/2_crop.png')
                    mask = cv2.imread(root+'/3_crop.png',0)
                    res2 = cv2.bitwise_or(img1,img2,mask = mask)
                    names = root.split("\\")
                    image_path = out_dir+names[0]+"_"+names[1]+"_"+names[2]+"_mask.png"
    #                 print("image path:",image_path)
                    cv2.imwrite(image_path,res2)
    #             image_name = root+"_mask.png"
    #             cv2.imwrite('or.png',res2)
                except:
                    continue        
        


def gather_binvox():
    directory = "Generated_Foraminifera"
    out_dir = "./binvox_files/"

    for root, dirs, files in os.walk(directory):
        for filename in files:
           
            if filename.endswith(".binvox"): 
                try:
                    print("root:",root,"dirs:",dirs,"files:",files)
                    binvox_path = root+"\\"+filename
                    print("binvox path:",binvox_path)
                    names = root.split("\\")
                    print("names:",names)
                    out_path = out_dir+names[0]+"_"+names[1]+"_"+filename
                    print("out_path:",out_path)
                    copyfile(binvox_path, out_path)
                except:
                    continue

def map_png_to_binvox(filename):
    parts = filename.split("_")
    s = parts[0]
    num = s[-1]
    s2 = s[:-1]
    binvox_filename = "Generated_Foraminifera_"+s2+"_"+num+"_"+parts[1]+"_"+parts[2]+".binvox"
    return(binvox_filename)
    
def images_to_numpy_array():
    mask_directory = "mask_images"
    binvox_directory = "binvox_files"
    x = [[]] # 3d images
    y = [[]] # 2.5 D images
    print("x:",x,"y:",y)
    for root, dirs, files in os.walk(mask_directory):
        for filename in files:
            if filename.endswith(".png"): 
                yi = cv2.imread(root+"//"+filename)
                y.append(yi)
                binvox_filename = map_png_to_binvox(filename)
                with open(binvox_directory+"//"+binvox_filename, 'rb') as f:
                    model = binvox_rw.read_as_3d_array(f)
                    #print("model:",model,"dim:",model.dims,"scale:",model.scale,"data:",model.data)
                    num_array = model.data.astype(int)
                    #print("num array:",num_array)
                    x.append(num_array)
                    
#     for root, dirs, files in os.walk(binvox_directory):
#         for filename in files:
#             if filename.endswith(".binvox"): 
#                 with open(root+"//"+filename, 'rb') as f:
#                     print("filename:",root+"/"+filename)
#                     model = binvox_rw.read_as_3d_array(f)
#                     #print("model:",model,"dim:",model.dims,"scale:",model.scale,"data:",model.data)
#                     num_array = model.data.astype(int)
#                     #print("num array:",num_array)
#                     x.append(num_array)
                    
    
    x.pop(0)
    y.pop(0)
    
    np_arr_y = np.array(y)
    np_arr_x = np.array(x)
    print("x shape:",np_arr_x.shape)
    print("y shape:",np_arr_y.shape)
#     print("fist y:",y[0])
#     print("sec y:",y[1])
    return np_arr_x, np_arr_y

    
data_X, data_Y = images_to_numpy_array()


x: [[]] y: [[]]
x shape: (1499, 64, 64, 64)
y shape: (1499, 320, 320, 3)


In [79]:
# model = encoder()
# model.compile(loss=keras.losses.kullback_leibler_divergence,
#               optimizer=keras.optimizers.SGD(lr=0.01),
#               metrics=['accuracy'])
# print(model.summary())

# model.fit(x_train, y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=(x_test, y_test),
#           callbacks=[history])
batch_size = 20
noise = np.random.normal(0,1,size=[batch_size, 200])
noise = np.reshape(noise,(20, 1,1,1,-1))
print(noise.shape)
    
def build_generator():
    model = Sequential()
    model.add(Conv3DTranspose(512, kernel_size=(4,4,4), strides=(1,1,1),input_shape=(1,1,1,200)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Conv3DTranspose(256,kernel_size=(4,4,4), strides=(2,2,2), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Conv3DTranspose(128, kernel_size=(4,4,4),strides=(2,2,2), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Conv3DTranspose(64, kernel_size=(4,4,4), strides=(2,2,2),padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Conv3DTranspose(1, kernel_size=(4,4,4), strides=(2,2,2), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Activation('sigmoid'))
    return model

GM = build_generator()
print(GM.summary())   
pred = GM.predict(noise)
print(pred.shape)

(20, 1, 1, 1, 200)


ValueError: Input 0 is incompatible with layer conv3d_transpose_6: expected ndim=5, found ndim=6

In [10]:
def sampling(args):
    """Reparameterization trick by sampling fr an isotropic unit Gaussian.

    # Arguments
        args (tensor): mean and log of variance of Q(z|X)

    # Returns
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [55]:
batch_size = 20
noise = np.random.normal(0,1,size=[batch_size, 307200])
noise = np.reshape(noise,(20, 320,320,3))
print(noise.shape)

def encoder():
    input_shape=(320,320,3)
    inputs = Input(shape=input_shape, name='encoder_input')
    x = Conv2D(64, kernel_size=(11, 11), strides=(5, 5), padding ='same')(inputs)
    x = BatchNormalization()(x)
    x= Activation('relu')(x)
    
    x = Conv2D(128, kernel_size=(5, 5), strides=(2, 2), padding ='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x= Conv2D(256, kernel_size=(5, 5), strides=(2, 2), padding ='same')(x)
    x= BatchNormalization()(x)
    x= Activation('relu')(x)
    
    x=Conv2D(512, kernel_size=(5, 5), strides=(2, 2), padding ='same')(x)
    x=BatchNormalization()(x)
    x=Activation('relu')(x)
    
    x=Conv2D(400, kernel_size=(8, 8), strides=(1, 1))(x)
    x=BatchNormalization()(x)
    x=Activation('relu')(x)
    
    z_mean = Dense(200)(x)
    z_log_var = Dense(200)(x)
    
    z = Lambda(sampling, output_shape=(200,), name='z')([z_mean, z_log_var])
    model = Model(inputs, z, name='encoder')
    return model

EM = encoder()
print(EM.summary())
  
pred = EM.predict(noise)
#print(pred)

(20, 320, 320, 3)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 320, 320, 3)  0                                            
__________________________________________________________________________________________________
conv2d_26 (Conv2D)              (None, 64, 64, 64)   23296       encoder_input[0][0]              
__________________________________________________________________________________________________
batch_normalization_41 (BatchNo (None, 64, 64, 64)   256         conv2d_26[0][0]                  
__________________________________________________________________________________________________
activation_34 (Activation)      (None, 64, 64, 64)   0           batch_normalization_41[0][0]     
___________________________________________________________________________________________

In [49]:
batch_size = 20
noise = np.random.normal(0,1,size=[batch_size, 262144])
noise = np.reshape(noise,(20,64,64,64,1))
print(noise.shape)
    
def build_discriminator():
    model = Sequential()
    model.add(Conv3D(64, kernel_size=(4,4,4), strides=(2,2,2),input_shape=(64, 64, 64,1), padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv3D(128,kernel_size=(4,4,4), strides=(2,2,2), padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv3D(256, kernel_size=(4,4,4),strides=(2,2,2), padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv3D(512, kernel_size=(4,4,4), strides=(2,2,2),padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv3D(1, kernel_size=(4,4,4), strides=(1,1,1)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Activation('sigmoid'))
    return model

DM = build_discriminator()
print(DM.summary())   
pred = DM.predict(noise)
print(pred.shape)

(20, 64, 64, 64, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_6 (Conv3D)            (None, 32, 32, 32, 64)    4160      
_________________________________________________________________
batch_normalization_31 (Batc (None, 32, 32, 32, 64)    256       
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 32, 32, 32, 64)    0         
_________________________________________________________________
conv3d_7 (Conv3D)            (None, 16, 16, 16, 128)   524416    
_________________________________________________________________
batch_normalization_32 (Batc (None, 16, 16, 16, 128)   512       
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 16, 16, 16, 128)   0         
_________________________________________________________________
conv3d_8 (Conv3D)            (None, 8, 8, 8, 256)      2

In [50]:
EDGModel = Sequential()
EDGModel.add(EM)
EDGModel.add(Reshape((1,1,1,200)))
EDGModel.add(GM)
EDGModel.add(DM)
EDGModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder (Model)              (None, 200)               17598432  
_________________________________________________________________
reshape_1 (Reshape)          (None, 1, 1, 1, 200)      0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 64, 64, 64, 1)     17572549  
_________________________________________________________________
sequential_7 (Sequential)    (None, 1, 1, 1, 1)        11051717  
Total params: 46,222,698
Trainable params: 46,216,134
Non-trainable params: 6,564
_________________________________________________________________


In [51]:
AMModel = Sequential()
AMModel.add(GM)
AMModel.add(DM)
AMModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_1 (Sequential)    (None, 64, 64, 64, 1)     17572549  
_________________________________________________________________
sequential_7 (Sequential)    (None, 1, 1, 1, 1)        11051717  
Total params: 28,624,266
Trainable params: 28,620,422
Non-trainable params: 3,844
_________________________________________________________________


In [16]:
def KL_loss(self, mu, log_var):
        return -0.5 * K.sum(1 + log_var - tf.pow(mu, 2) - tf.exp(log_var))

In [17]:
def sample_z(self, mu, log_var):
        eps = K.random_normal(shape=K.shape(mu))
        return mu + K.exp(log_var / 2) * eps

In [18]:
def NLLNormal(self, pred, target):

        c = -0.5 * K.log(2 * np.pi)
        multiplier = 1.0 / (2.0 * 1)
        tmp = K.square(pred - target)
        tmp *= -multiplier
        tmp += c

        return tmp

In [20]:
print(data_X.shape)
print(data_Y.shape)

(1499, 64, 64, 64)
(1499, 320, 320, 3)


In [22]:
data_X[0:20].shape


(20, 64, 64, 64)

In [78]:
# def custom_loss(encoder_output):
#   def loss(y_true, y_pred):
#     noise = np.random.normal(0,1,size=[batch_size, 200])
        
#     #200 is the channel size since we have 200 as the number of features.
#     noise = np.reshape(noise,(batch_size,1,1,1,200))
#     loss1 = kullback_leibler_divergence(noise, encoder_output)
#     loss2 = mean_squared_error(y_true, y_pred)
#     return loss1+loss2
#   return loss

def noise_loss(ytrue, ypred):
  return log(1-ypred)
  
def real_loss(ytrue, ypred):
  return mean_squared_error(ytrue, ypred)

#Compile GM model to train discriminator
GM.compile(loss= 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])
DM.compile(loss= 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#EDGModel.compile(loss="kullback_leibler_divergence", optimizer='adam', metrics=['accuracy'])
#AM.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#Compile EG model to train encoder
input_shape2=(320,320,3)
EGModel = Sequential()
EGModel.add(EM)
EGModel.add(Reshape((1,1,1,200)))
output1 = EGModel.add(GM)
EGModel.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# #compile AM model to train generator.
# input_shape2=(320,320,3)
# input_shape3=(1,1,1,200)
# input1 = Input(shape=input_shape2)
# input2 = Input(shape=input_shape3)
# #Part taking real input in
# output1 = EM(input1)
# print(output1.shape)
# output1 = np.reshape(np.array(output1),(20,1,1,1,200))
# output2 = GM()(output1)
# #Part which takes noise in
# output3 = GM(input2)
# output4 = DM()(output3)
# losses = [real_loss, noise_loss]
# AModel = Model(input =[input1, input2], output=[output2, output4])
# AMmodel.compile(optimizer='adam', loss=losses, metrics=["accuracy"])

def train_model():
    batch_size = 20
    
    real_2d_images = data_Y
    real_2d_images = real_2d_images[:-19]
    real_3d_images = data_X
    real_3d_images = real_3d_images[:-19]
    for i in range(1):
        
        start = 0
        print(len(real_2d_images))
        for j in range(len(real_2d_images)//batch_size - batch_size):
#             noise = np.random.normal(0,1,size=[batch_size, 200])

#             #200 is the channel size since we have 200 as the number of features.
#             noise = np.reshape(noise,(batch_size,1,1,1,200))

#             # Expect 20 64*64*64 images.
#             fake_models = GM.predict(noise)
#             print("fake", fake_models.shape)

#             noise2 = np.zeros([20, 320,320,3])
#             print(noise2.shape)
#             a = EM.predict(noise2)
#             print(real_3d_images.shape)

            real_2d_images = np.reshape(real_2d_images[start:start+batch_size], (batch_size, 320, 320, 3))
            real_3d_images = np.reshape(real_3d_images[start:start+batch_size], (batch_size, 64, 64, 64, 1))
#             # STEP 1: Update the discriminator
#             #x = concatenate 20 real models and 20 fake models
#             #y = 1 for the real models and 0 for the fake models
#             y_noise = np.zeros([batch_size])  

#             y_noise = np.reshape(y_noise,(batch_size,1,1,1,1))
#             print(y_noise.shape)
#             d_output_z = DM.train_on_batch(fake_models, y_noise)
#             print("Loss on noise ofDM", d_output_z)
#             y_real = np.ones([batch_size])        
#             y_real = np.reshape(y_real,(batch_size,1,1,1,1))
#             d_output_x = DM.train_on_batch(real_3d_images, y_real)  
#             print("Loos on real images of DM", d_output_x)
            # STEP 2: Update the encoder
            EGModel.train_on_batch(real_2d_images, real_3d_images)
            start = start+batch_size
#         # STEP 3: Update the generator
#         # now trianing with 1 for noise.
#         y_noise = np.ones([batch_size, 1])  
#         AMModel.train_on_batch([real_2d_images, noise],[real_3d_images,y_noise])

train_model()

1480


InvalidArgumentError: Input to reshape is a tensor with 80000 values, but the requested shape has 4000
	 [[{{node reshape_25/Reshape}}]]